In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 6.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=11160aa7247f0e647ee20d73cc4a625773465cc1d1b990db26a2fff3e67a32f1
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 7.5 MB/s 
     |████████████████████████████████| 769 kB 1.2 MB/s 
     |████████████████████████████████| 3.0 MB 65.5 MB/s 
     |████████████████████████████████| 895 kB 75.0 MB/s 


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-lbs0pt_p
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-lbs0pt_p
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=57b485c27c7a6753e21df3c54b8112934cccef743bc8f625d6bd803149b31671
  Stored in directory: /tmp/pip-ephem-wheel-cache-prde75xx/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [7]:
#구글 코랩 사용 시 드라이버 연결 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
total_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/빅콘테스트/total_dataset.csv')
#total_dataset = pd.read_csv('./total_dataset.csv') 1번코드에서 만들어진 total_dataset.csv로 저장경로 지정해주기 

In [9]:
total_dataset.sample(n=10)

,Unnamed: 0,sentence,values,sentence_len
25548,25548,특히 국내간사기관 지정교육기관 검증기관 대한민국지속가능성지수 운영기관 ...,A,99
23326,23326,환경경영 환경 기후변화 대응 안전 품질 임직원 협력사 지역사회 지배구...,A,292
39415,39415,레일솔루션 사업부문은 철도차량 기술기준에서 요구하는 안전 년 최초로 를 도입한 후...,B+,353
21350,21350,그린수소 생산 실증 국책과제 참여과제명과제기간동해 태양광 연계형 시스템 설계 및 ...,A,229
21844,21844,우리는 연결재무제표 감사와 관련된 대한민국의 윤리적 요구사항에 따라 회사로부터 독...,A,84
31801,31801,이를 통해 전자의 안전문화 지수를 년,B+,22
33134,33134,억원 사 컨설팅,B+,11
18764,18764,재팬 네이버 재팬 라이브도어 등 개 법인 재팬으로 통합,A,34
14527,14527,정부가 화석연료에 대한 의존도를 낮추기 위해 기후변화 대응에너지로 를 주목하고...,A,92
13861,13861,전자투표제 도입전자투표제는 기업이 전자투표시스템에 주주 명부와 주주총회 의안을 등록...,A,98


In [10]:
total_dataset.loc[(total_dataset['values'] == "A+"), 'values'] = 0 # A+ => 0
total_dataset.loc[(total_dataset['values'] == "A"), 'values'] = 1  # A => 1
total_dataset.loc[(total_dataset['values'] == "B+"), 'values'] = 2 # B+ => 2
total_dataset.loc[(total_dataset['values'] == "B"), 'values'] = 3  # B => 3
total_dataset.loc[(total_dataset['values'] == "C"), 'values'] = 4  # C => 4
data_list = []
for q, label in zip(total_dataset['sentence'], total_dataset['values'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [11]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['     금융그룹 지속가능경영보고서  보고서 개요  보고 범위  금융그룹은 국민은행이 년부터 발간해 온 지속가능경영보고서를 본 보고서는 금융그룹 지주 및 개 계열사의 지속가능경영 활동을 담고 년부터 그룹 차원으로 확대하여 매년 발간하고 있습니다', '0']
['  채권은 친환경 사업과 사회적 가치 창출 목적으로 발행 자금을 사용해야 하는 채권으로 그간 국내에서는 금융기관과 공기업 제조기업 위주로  채권을 발행해왔으며  업계에서는 가 최초로  채권을 발행하는 기업입니다', '0']
['기업개요년 월 일 기준기업명업종㈜제일제당식품 바이오본사 주소서울특별시 중구 동호로  제일제당센터         식품사업우수한 한식을 세계에 널리 전파하여 - 식문화를 선도하겠습니다', '1']
[' 학점교류 인턴십을 운영해 산학협력대학 반도체 관련 학과 학생들에게 회사 생활 경험과 직무 탐색의 기회를 제공하고 카이스트 반도체공학프로그램 고려대 반도체공학과 고려대 대학원 반도체시스템공학과 한양대 나노반도체공학과 등의 계약학과를 통해 공학우수인재를 선발하고 지원합니다', '1']
['  용인 민자 하수처리장지역사회 상생 지원을 통한 사회공헌 실천 - 도스보카스 정유 프로젝트도스보카스 정유 프로젝트는 멕시코 동부 타바스코 주州 도스보카스  지역에서 진행되고 있는 프로젝트로 하루 만 배럴의 원유 생산설비를 건설하는 사업입니다', '1']
['\t년\t어업계\t발전\t상생협약을\t맺은\t이래로\t사업장\t인근\t저수지와\t호수\t등\t자연환경의\t생물\t다양성\t및\t생태\t환경의\t보존을\t위해\t치어\t방류\t실개천\t복원\t등의\t환경\t보존\t활동들을\t수행해오고\t있습니다', '1']
['본 사    충청남도 당진시 면천면 면천로    -   -서 울   서울특별시 서초구 효령로     -   -', '4']


In [12]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [13]:
print(len(dataset_train))
print(len(dataset_test))

34479
11493


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [15]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [16]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [18]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [19]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [20]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/539 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.6135236024856567 train acc 0.265625
epoch 1 batch id 201 loss 1.2946374416351318 train acc 0.4145677860696517
epoch 1 batch id 401 loss 1.0899158716201782 train acc 0.46376246882793015
epoch 1 train acc 0.49523102277661546


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6545115427927928


  0%|          | 0/539 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4224558472633362 train acc 0.828125
epoch 5 batch id 201 loss 0.6457486152648926 train acc 0.8159203980099502
epoch 5 batch id 401 loss 0.5395606756210327 train acc 0.827930174563591
epoch 5 train acc 0.8331271217384439


  0%|          | 0/180 [00:00<?, ?it/s]

epoch 5 test acc 0.6631052927927928


In [21]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("A+")
            elif np.argmax(logits) == 1:
                test_eval.append("A")
            elif np.argmax(logits) == 2:
                test_eval.append("B+")
            elif np.argmax(logits) == 3:
                test_eval.append("B")
            elif np.argmax(logits) == 4:
                test_eval.append("C")


        print(">> 입력하신 내용의 ESG등급은 " + test_eval[0] + " 입니다.")

using cached model


In [ ]:
end = 1
while end == 1 :
    sentence = input("등급 확인하고 싶은 내용을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

등급 확인하고 싶은 내용을 입력해주세요 : 인권


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용의 ESG등급은 A 입니다.


등급 확인하고 싶은 내용을 입력해주세요 : 탄소배출 제로
>> 입력하신 내용의 ESG등급은 A+ 입니다.


등급 확인하고 싶은 내용을 입력해주세요 : 지역사회 관계
>> 입력하신 내용의 ESG등급은 A 입니다.


등급 확인하고 싶은 내용을 입력해주세요 : 부패
>> 입력하신 내용의 ESG등급은 B 입니다.


등급 확인하고 싶은 내용을 입력해주세요 : ESG위원회 구성
>> 입력하신 내용의 ESG등급은 A 입니다.


